# Libraries

In [1]:
from datetime import datetime

import pandas as pd
import numpy as np
import geopandas as gpd
from google.cloud import storage
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from pyquadkey2 import quadkey as qk

# Functions

In [2]:
def query_osm_overpass(area_name, key, value):

    nominatim = Nominatim()
    areaId = nominatim.query(area_name).areaId()
    overpass = Overpass()
    query = overpassQueryBuilder(area=areaId, elementType='way', selector=f'"{key}"="{value}"', out='body', includeGeometry=True)
    result = overpass.query(query, timeout=120)
    
    return result



def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

def merge_with_connectivity(out, ookla):
    out = out[~out.lat.isna()]
    out['quadkey'] = out.apply(make_quadkey, axis=1)
    out['quadkey'] = out['quadkey'].astype('str')
    connectivity = ookla[ookla.quadkey.isin(out['quadkey'])]
    connectivity = connectivity.set_index('quadkey')
    tmp = out.set_index('quadkey')
    tmp.index = tmp.index.astype('str')
    tmp = tmp.join(connectivity, how='left')
    tmp = tmp.reset_index().drop_duplicates(
        subset=['name', 'quadkey'],
        keep='last').reset_index(drop=True)
    # return connectivity
    return tmp

def make_quadkey(row):
    return qk.from_geo((row['lat'], row['lon']), 16)

def make_details(element):
    tags = element.tags()
    if 'name' in tags:
        out = dict()
        out['amenity'] = tags['amenity']
        out['name'] = tags['name']
        if 'lat' in tags:
            out['lat'] = tags['lat']
            out['lon'] = tags['lon']
        elif bool(element.centerLat()):
            out['lat'] = element.centerLat()
            out['lon'] = element.centerLon()
        elif bool(element.geometry()):
            out['lat'] = element.geometry()['coordinates'][0][0][1]
            out['lon'] = element.geometry()['coordinates'][0][0][0]
        else:
            out['lat'] = None
            out['lon'] = None
        return out
    else:
        return None
    
def filter_elements(result):
    elements = list()
    for element in result.elements():
        elements.append(make_details(element))
    filtered = [element for element in elements if element is not None]
    return filtered

In [3]:
poly = gpd.read_file('D:\Work\WB\LDT\countries\SRB\shapefiles\gadm41_SRB_2.json')
poly.head()

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,SRB.1.1_1,SRB,Serbia,SRB.1_1,Borski,Борски,Bor,NA,Бор,Opštine,Town|Municipal,NA,NA,"MULTIPOLYGON (((21.9554 44.1779, 21.9912 44.17..."
1,SRB.1.2_1,SRB,Serbia,SRB.1_1,Borski,Борски,Kladovo,Claudia|Kladowo,Кладово,Opštine,Town|Municipal,NA,NA,"MULTIPOLYGON (((22.6492 44.5411, 22.644 44.545..."
2,SRB.1.3_1,SRB,Serbia,SRB.1_1,Borski,Борски,Majdanpek,NA,Мајданпек,Opštine,Town|Municipal,NA,NA,"MULTIPOLYGON (((22.1915 44.4985, 22.1967 44.51..."
3,SRB.1.4_1,SRB,Serbia,SRB.1_1,Borski,Борски,Negotin,NA,Неготин,Opštine,Town|Municipal,NA,NA,"MULTIPOLYGON (((22.6851 44.2099, 22.6853 44.20..."
4,SRB.2.1_1,SRB,Serbia,SRB.2_1,Braničevski,Браничевски,Golubac,Galambóc|Golumbácz|Güvercinlik,Голубац,Opštine,Town|Municipal,NA,NA,"MULTIPOLYGON (((21.8695 44.6506, 21.8972 44.64..."


In [139]:
result = query_osm_overpass('Serbia', 'amenity', 'hospital')
filtered = filter_elements(result)
out = pd.DataFrame(filtered)
out.to_csv('D:\Work\WB\LDT\countries\SRB\\raw_data\hospitals.csv', index=False)

result = query_osm_overpass('Serbia', 'amenity', 'school')
filtered = filter_elements(result)
out = pd.DataFrame(filtered)
out.to_csv('D:\Work\WB\LDT\countries\SRB\\raw_data\schools.csv', index=False)


,amenity,name,lat,lon
0,school,Гимназија Вељко Петровић,45.771823,19.113606
1,school,Средња техничка школа Сомбор (I зграда),45.773831,19.112550
2,school,Средња техничка школа Сомбор (II зграда),45.772993,19.110734
3,school,Основна школа Доситеј Обрадовић,45.773171,19.111040
4,school,Средња техничка школа Сомбор (III зграда),45.774006,19.109396
...,...,...,...,...
1224,school,Prapaqan School,42.559229,20.340259
1225,school,"Основна школа ""Доситеј Обрадовић"" - Подручно о...",45.088961,19.833926
1226,school,Карловачка гимназија,45.201411,19.933122
1227,school,ОШ „Светитељ Сава”,44.547530,20.832829


In [4]:
hospitals = pd.read_csv('D:\Work\WB\LDT\countries\SRB\\raw_data\hospitals.csv')
schools = pd.read_csv('D:\Work\WB\LDT\countries\SRB\\raw_data\schools.csv')

assets = pd.concat([hospitals, schools], axis=0).reset_index(drop=True)
assets.head()

,amenity,name,lat,lon
0,hospital,ГАК Народни фронт,44.809618,20.460292
1,hospital,Неуропсихијатрија,45.768977,19.106787
2,hospital,Клинички центар Војводине,45.254037,19.823653
3,hospital,Специјална болница за реуматске болести,45.249292,19.828312
4,hospital,Клиничко-болнички центар „Звездара“,44.800520,20.501743


In [5]:
sv_acc = 'D:\Work\WB\LDT\credentials\wb-ldt-948953b71056.json'

ookla_fixed = pd.read_parquet('gcs://wb-ldt/RS/raw-data/geospatial/internet/2022_combined_fixed.parquet',
                        storage_options={"token": sv_acc})
ookla_fixed.quadkey = ookla_fixed.quadkey.astype('str')

# ookla_mobile = pd.read_parquet('gcs://wb-ldt/RS/raw-data/geospatial/internet/2022_combined_mobile.parquet',
#                         storage_options={"token": sv_acc})
# ookla_mobile.quadkey = ookla_mobile.quadkey.astype('str')

In [6]:
ookla_fixed.head()

,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,0013230212122122,"POLYGON((-105.97412109375 79.8481885833186, -1...",544197.000000,269445.0,19.000000,1,1
1,0022133222312233,"POLYGON((-160.032348632812 70.6435894914449, -...",125266.000000,15523.0,127.000000,1,1
2,0022133222312322,"POLYGON((-160.02685546875 70.6435894914449, -1...",66682.666667,8126.0,95.666667,22,5
3,0022133222330013,"POLYGON((-160.032348632812 70.6399478155463, -...",223427.000000,23185.0,91.000000,1,1
4,0022133222330023,"POLYGON((-160.043334960938 70.6363054807905, -...",125703.000000,14623.0,270.000000,3,1


In [8]:
assets_merged = gpd.GeoDataFrame(merge_with_connectivity(assets, ookla_fixed))
geometry = gpd.points_from_xy(assets_merged['lon'], 
                              assets_merged['lat'], 
                              crs='EPSG:4326')
assets_merged = assets_merged.set_geometry(geometry)
assets_merged = assets_merged.to_crs(poly.crs)
assets_joined = assets_merged.sjoin(poly[['GID_2', 'geometry']], how='inner')


assets_joined.head()

,quadkey,amenity,name,lat,lon,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,GID_2
0,1202331012001322,hospital,ГАК Народни фронт,44.809618,20.460292,"POLYGON((20.45654296875 44.8130187406128, 20.4...",100034.00,36584.50,7.75,1351.0,593.0,POINT (20.46029 44.80962),19,SRB.3.8_1
1,1202312312012332,hospital,Неуропсихијатрија,45.768977,19.106787,"POLYGON((19.105224609375 45.7713547044504, 19....",99504.25,32104.00,9.25,140.0,52.0,POINT (19.10679 45.76898),150,SRB.24.4_1
2,1202313222031202,hospital,Клинички центар Војводине,45.254037,19.823653,"POLYGON((19.8193359375 45.2555552778921, 19.82...",70243.50,23539.50,11.50,99.0,53.0,POINT (19.82365 45.25404),40,SRB.5.6_1
3,1202313222031221,hospital,Специјална болница за реуматске болести,45.249292,19.828312,"POLYGON((19.8248291015625 45.2516882561177, 19...",109038.75,30681.75,8.75,202.0,60.0,POINT (19.82831 45.24929),40,SRB.5.6_1
4,1202331012012120,hospital,Клиничко-болнички центар „Звездара“,44.800520,20.501743,"POLYGON((20.50048828125 44.8013268290486, 20.5...",144306.75,45416.25,8.50,361.0,102.0,POINT (20.50174 44.80052),19,SRB.3.8_1


# Indicators

## EU Broadband Target
* https://www.capacitymedia.com/article/29ot865vu8akt0k1irsow/news/european-union-to-set-100mbps-internet-targets-for-2025
* https://ec.europa.eu/newsroom/dae/document.cfm?doc_id=70030

In [9]:
assets_pass = assets_joined[assets_joined['avg_d_kbps'] >= 30000].groupby(['GID_2']).agg(assets_passed=('name', 'count')).reset_index()
assets_pass = assets_pass.merge(poly[['GID_2', 'NAME_2']], on=['GID_2'], how='right')
assets_pass['assets_passed'] = assets_pass['assets_passed'].fillna(0)

assets_total = assets_joined.groupby(['GID_2']).agg(assets_total=('name', 'count')).reset_index()
assets_total = assets_total.merge(poly[['GID_2', 'NAME_2']], on=['GID_2'], how='right')
assets_total['assets_total'] = assets_total['assets_total'].fillna(0)

pass_df = pd.merge(assets_pass, assets_total, on=['GID_2', 'NAME_2'], how='inner')
pass_df['pass_percentage'] = np.round(100 * pass_df['assets_passed'] / pass_df['assets_total'], 2)
pass_df['pass_percentage'] = pass_df['pass_percentage'].fillna(0)
pass_df['year'] = 2022
pass_df

,GID_2,assets_passed,NAME_2,assets_total,pass_percentage,year
0,SRB.1.1_1,7.0,Bor,10.0,70.00,2022
1,SRB.1.2_1,3.0,Kladovo,3.0,100.00,2022
2,SRB.1.3_1,3.0,Majdanpek,6.0,50.00,2022
3,SRB.1.4_1,0.0,Negotin,3.0,0.00,2022
4,SRB.2.1_1,0.0,Golubac,0.0,0.00,2022
...,...,...,...,...,...,...
156,SRB.25.6_1,1.0,Požega,5.0,20.00,2022
157,SRB.25.7_1,1.0,Priboj,2.0,50.00,2022
158,SRB.25.8_1,5.0,Prijepolje,6.0,83.33,2022
159,SRB.25.9_1,0.0,Sjenica,1.0,0.00,2022


In [10]:
pass_df.describe()

,assets_passed,assets_total,pass_percentage,year
count,161.000000,161.000000,161.000000,161.0
mean,4.664596,6.645963,50.787764,2022.0
std,10.702771,12.160083,40.515057,0.0
min,0.000000,0.000000,0.000000,2022.0
25%,0.000000,2.000000,0.000000,2022.0
50%,2.000000,3.000000,50.000000,2022.0
75%,5.000000,6.000000,93.750000,2022.0
max,107.000000,108.000000,100.000000,2022.0


## Average Download Speed

In [12]:
assets_gb = assets_joined.groupby(['GID_2']).agg({'avg_d_kbps': 'mean'}).reset_index()

assets_gb = assets_gb.merge(poly[['GID_2', 'NAME_2']], on=['GID_2'], how='right')
assets_gb['avg_d_kbps'] = assets_gb['avg_d_kbps'].fillna(0)
assets_gb['year'] = 2022
assets_gb

,GID_2,avg_d_kbps,NAME_2,year
0,SRB.1.1_1,52114.375000,Bor,2022
1,SRB.1.2_1,59009.416667,Kladovo,2022
2,SRB.1.3_1,48167.000000,Majdanpek,2022
3,SRB.1.4_1,18188.750000,Negotin,2022
4,SRB.2.1_1,0.000000,Golubac,2022
...,...,...,...,...
156,SRB.25.6_1,24896.187500,Požega,2022
157,SRB.25.7_1,29540.875000,Priboj,2022
158,SRB.25.8_1,31094.708333,Prijepolje,2022
159,SRB.25.9_1,20990.250000,Sjenica,2022


## Key Structures without Internet Access

In [13]:
null = assets_joined[assets_joined['avg_d_kbps'].isnull()].groupby(['GID_2']).agg(struc_null=('name', 'count')).reset_index()
compare = null.merge(poly, on=['GID_2'], how='right')
compare['struc_null'] = compare['struc_null'].fillna(0)

nnull = assets_joined[assets_joined['avg_d_kbps'].notnull()].groupby(['GID_2']).agg(struc_tot=('name', 'count')).reset_index()
compare = compare.merge(nnull, on=['GID_2'], how='left')
compare['struc_tot'] = compare['struc_tot'].fillna(0)
compare['null_perc'] = 100* compare['struc_null'] / compare['struc_tot']
compare['null_perc'] = compare['null_perc'].fillna(0)
compare['year'] = 2022
compare = compare[['GID_2', 'NAME_2', 'year', 'null_perc']]
compare

,GID_2,NAME_2,year,null_perc
0,SRB.1.1_1,Bor,2022,0.000000
1,SRB.1.2_1,Kladovo,2022,0.000000
2,SRB.1.3_1,Majdanpek,2022,20.000000
3,SRB.1.4_1,Negotin,2022,0.000000
4,SRB.2.1_1,Golubac,2022,0.000000
...,...,...,...,...
156,SRB.25.6_1,Požega,2022,25.000000
157,SRB.25.7_1,Priboj,2022,0.000000
158,SRB.25.8_1,Prijepolje,2022,0.000000
159,SRB.25.9_1,Sjenica,2022,0.000000


In [15]:
df = pd.merge(pass_df[['GID_2', 'NAME_2', 'year', 'pass_percentage']], assets_gb,
              on=['GID_2', 'NAME_2', 'year'], how='inner')
df = pd.merge(df, compare,
              on=['GID_2', 'NAME_2', 'year'], how='inner')

df

,GID_2,NAME_2,year,pass_percentage,avg_d_kbps,null_perc
0,SRB.1.1_1,Bor,2022,70.00,52114.375000,0.000000
1,SRB.1.2_1,Kladovo,2022,100.00,59009.416667,0.000000
2,SRB.1.3_1,Majdanpek,2022,50.00,48167.000000,20.000000
3,SRB.1.4_1,Negotin,2022,0.00,18188.750000,0.000000
4,SRB.2.1_1,Golubac,2022,0.00,0.000000,0.000000
...,...,...,...,...,...,...
156,SRB.25.6_1,Požega,2022,20.00,24896.187500,25.000000
157,SRB.25.7_1,Priboj,2022,50.00,29540.875000,0.000000
158,SRB.25.8_1,Prijepolje,2022,83.33,31094.708333,0.000000
159,SRB.25.9_1,Sjenica,2022,0.00,20990.250000,0.000000


In [16]:
df.to_csv('D:\Work\WB\LDT\countries\SRB\datasets/SRB_fixed_2022.csv', index=False)

## Cellular Download Speed

In [17]:
from shapely import wkt

In [18]:
sv_acc = 'D:\Work\WB\LDT\credentials\wb-ldt-948953b71056.json'

ookla_mobile = pd.read_parquet('gcs://wb-ldt/RS/raw-data/geospatial/internet/2022_combined_mobile.parquet',
                        storage_options={"token": sv_acc})
ookla_mobile.quadkey = ookla_mobile.quadkey.astype('str')

geometry = wkt.loads(ookla_mobile['tile'])
ookla_mobile = gpd.GeoDataFrame(ookla_mobile, geometry=geometry, crs=poly.crs)

In [19]:
ookla_mobile

,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,0022133222312322,"POLYGON((-160.02685546875 70.6435894914449, -1...",17660.000000,14932.500000,82.500000,4,3,"POLYGON ((-160.02686 70.64359, -160.02136 70.6..."
1,0022133222330023,"POLYGON((-160.043334960938 70.6363054807905, -...",19547.333333,14359.666667,107.666667,8,3,"POLYGON ((-160.04333 70.63631, -160.03784 70.6..."
2,0022133222330031,"POLYGON((-160.032348632812 70.6381267305321, -...",6790.000000,22363.000000,68.000000,1,1,"POLYGON ((-160.03235 70.63813, -160.02686 70.6..."
3,0022133222330032,"POLYGON((-160.037841796875 70.6363054807905, -...",14094.666667,14110.000000,156.333333,16,5,"POLYGON ((-160.03784 70.63631, -160.03235 70.6..."
4,0022133222330100,"POLYGON((-160.02685546875 70.6417687358462, -1...",7543.500000,12821.500000,87.500000,7,2,"POLYGON ((-160.02686 70.64177, -160.02136 70.6..."
...,...,...,...,...,...,...,...,...
8050644,3131120221002013,"POLYGON((169.118041992188 -46.5928439974274, 1...",54752.000000,26099.000000,59.000000,1,1,"POLYGON ((169.11804 -46.59284, 169.12354 -46.5..."
8050645,3131120221002122,"POLYGON((169.12353515625 -46.6003930373455, 16...",59200.000000,15101.000000,54.000000,1,1,"POLYGON ((169.12354 -46.60039, 169.12903 -46.6..."
8050646,3131120221020110,"POLYGON((169.134521484375 -46.6192610361715, 1...",53663.000000,31816.000000,41.000000,2,2,"POLYGON ((169.13452 -46.61926, 169.14001 -46.6..."
8050647,3131120221022002,"POLYGON((169.1015625 -46.6532068712266, 169.10...",29173.000000,30286.000000,62.000000,1,1,"POLYGON ((169.10156 -46.65321, 169.10706 -46.6..."


In [1]:
mobile_df = gpd.sjoin(ookla_mobile, poly[['GID_2', 'NAME_2', 'geometry']], how='inner')
mobile_gb = mobile_df.groupby(['GID_2', 'NAME_2']).agg({'avg_d_kbps': 'mean'}).reset_index()
mobile_gb['year'] = 2022
mobile_gb

In [21]:
mobile_gb.to_csv('D:\Work\WB\LDT\countries\SRB\datasets/SRB_mobile_2022.csv', index=False)

# Raw Data Processing

In [175]:
import pandas as pd
df = pd.read_parquet('D:\Work\WB\LDT\countries\SRB\\raw_data\internet\\2022-01-01_performance_fixed_tiles.parquet')
df.head()

,quadkey,tile,tile_x,tile_y,avg_d_kbps,avg_u_kbps,avg_lat_ms,avg_lat_down_ms,avg_lat_up_ms,tests,devices
0,0022332203013331,"POLYGON((-162.603149414062 66.8999063523575, -...",-162.6004,66.8988,50519,24352,18,NaN,NaN,18,5
1,0022332203013333,"POLYGON((-162.603149414062 66.8977510769183, -...",-162.6004,66.8967,10057,3988,9,NaN,NaN,2,1
2,0022332203031110,"POLYGON((-162.608642578125 66.8955956114071, -...",-162.6059,66.8945,900297,75822,7,NaN,NaN,2,1
3,0022332203031111,"POLYGON((-162.603149414062 66.8955956114071, -...",-162.6004,66.8945,370955,62117,8,NaN,NaN,4,3
4,0022332203031113,"POLYGON((-162.603149414062 66.8934399558101, -...",-162.6004,66.8924,323725,37943,23,NaN,NaN,3,2


In [10]:
df.drop(['tile', 'tile_x', 'tile_y', 'avg_lat_down_ms', 'avg_lat_up_ms'], axis=1)

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,0022332203013331,50519,24352,18,18,5
1,0022332203013333,10057,3988,9,2,1
2,0022332203031110,900297,75822,7,2,1
3,0022332203031111,370955,62117,8,4,3
4,0022332203031113,323725,37943,23,3,2
...,...,...,...,...,...,...
6780810,3131120300000300,49545,23975,13,4,1
6780811,3200202311023123,26478,108402,570,1,1
6780812,3220103201010202,7110,33881,244,1,1
6780813,3300323003310001,30601,9786,3,1,1


In [180]:
from glob import glob
paths = glob('D:\Work\WB\LDT\countries\SRB\\raw_data\internet\\*_mobile_tiles*')

dfs = []
for path in paths:
    print(path)
    df = pd.read_parquet(path)
    df = df.drop(['tile_x', 'tile_y', 'avg_lat_down_ms', 'avg_lat_up_ms'], axis=1)
    dfs.append(df)

full = pd.concat(dfs, axis=0).reset_index(drop=True)

D:\Work\WB\LDT\countries\SRB\raw_data\internet\2022-01-01_performance_mobile_tiles.parquet
D:\Work\WB\LDT\countries\SRB\raw_data\internet\2022-04-01_performance_mobile_tiles.parquet
D:\Work\WB\LDT\countries\SRB\raw_data\internet\2022-07-01_performance_mobile_tiles.parquet
D:\Work\WB\LDT\countries\SRB\raw_data\internet\2022-10-01_performance_mobile_tiles.parquet


In [181]:
full

,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,0022133222312322,"POLYGON((-160.02685546875 70.6435894914449, -1...",19110,7891,77,3,2
1,0022133222330023,"POLYGON((-160.043334960938 70.6363054807905, -...",21870,11875,83,2,1
2,0022133222330032,"POLYGON((-160.037841796875 70.6363054807905, -...",14567,14691,75,9,2
3,0022133222330100,"POLYGON((-160.02685546875 70.6417687358462, -1...",5468,9886,83,1,1
4,0022133222330102,"POLYGON((-160.02685546875 70.6399478155463, -1...",24311,16243,72,1,1
...,...,...,...,...,...,...,...
15732682,3131120211221222,"POLYGON((169.8486328125 -46.2824277013447, 169...",61559,31302,38,1,1
15732683,3131120211232103,"POLYGON((169.920043945312 -46.2900198717295, 1...",1146,873,49,1,1
15732684,3131120212300111,"POLYGON((169.667358398438 -46.4378568950242, 1...",65504,15991,40,1,1
15732685,3131120212300123,"POLYGON((169.656372070312 -46.4492124038526, 1...",96368,16433,53,1,1


In [182]:
full_gb = full.groupby(['quadkey', 'tile']).agg({'avg_d_kbps': 'mean',
                                         'avg_u_kbps': 'mean',
                                         'avg_lat_ms': 'mean',
                                         'tests': 'sum', 
                                         'devices': 'sum'}).reset_index()
full_gb


,quadkey,tile,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices
0,0022133222312322,"POLYGON((-160.02685546875 70.6435894914449, -1...",17660.000000,14932.500000,82.500000,4,3
1,0022133222330023,"POLYGON((-160.043334960938 70.6363054807905, -...",19547.333333,14359.666667,107.666667,8,3
2,0022133222330031,"POLYGON((-160.032348632812 70.6381267305321, -...",6790.000000,22363.000000,68.000000,1,1
3,0022133222330032,"POLYGON((-160.037841796875 70.6363054807905, -...",14094.666667,14110.000000,156.333333,16,5
4,0022133222330100,"POLYGON((-160.02685546875 70.6417687358462, -1...",7543.500000,12821.500000,87.500000,7,2
...,...,...,...,...,...,...,...
8050644,3131120221002013,"POLYGON((169.118041992188 -46.5928439974274, 1...",54752.000000,26099.000000,59.000000,1,1
8050645,3131120221002122,"POLYGON((169.12353515625 -46.6003930373455, 16...",59200.000000,15101.000000,54.000000,1,1
8050646,3131120221020110,"POLYGON((169.134521484375 -46.6192610361715, 1...",53663.000000,31816.000000,41.000000,2,2
8050647,3131120221022002,"POLYGON((169.1015625 -46.6532068712266, 169.10...",29173.000000,30286.000000,62.000000,1,1


In [183]:
full_gb.to_parquet('D:\Work\WB\LDT\countries\SRB\\raw_data\internet\\2022_combined_mobile.parquet', index=False)

In [173]:
test = pd.read_parquet('D:\Work\WB\LDT\countries\SRB\\raw_data\internet\\2022-01-01_performance_fixed_tiles.parquet')
test

,quadkey,tile,tile_x,tile_y,avg_d_kbps,avg_u_kbps,avg_lat_ms,avg_lat_down_ms,avg_lat_up_ms,tests,devices
0,0022332203013331,"POLYGON((-162.603149414062 66.8999063523575, -...",-162.6004,66.8988,50519,24352,18,NaN,NaN,18,5
1,0022332203013333,"POLYGON((-162.603149414062 66.8977510769183, -...",-162.6004,66.8967,10057,3988,9,NaN,NaN,2,1
2,0022332203031110,"POLYGON((-162.608642578125 66.8955956114071, -...",-162.6059,66.8945,900297,75822,7,NaN,NaN,2,1
3,0022332203031111,"POLYGON((-162.603149414062 66.8955956114071, -...",-162.6004,66.8945,370955,62117,8,NaN,NaN,4,3
4,0022332203031113,"POLYGON((-162.603149414062 66.8934399558101, -...",-162.6004,66.8924,323725,37943,23,NaN,NaN,3,2
...,...,...,...,...,...,...,...,...,...,...,...
6780810,3131120300000300,"POLYGON((170.17822265625 -46.0884717957759, 17...",170.1810,-46.0904,49545,23975,13,NaN,NaN,4,1
6780811,3200202311023123,"POLYGON((2.5323486328125 -72.0112484218371, 2....",2.5351,-72.0121,26478,108402,570,NaN,NaN,1,1
6780812,3220103201010202,"POLYGON((14.501953125 -79.9407167525038, 14.50...",14.5047,-79.9412,7110,33881,244,NaN,NaN,1,1
6780813,3300323003310001,"POLYGON((104.683227539062 -73.3782147793946, 1...",104.6860,-73.3790,30601,9786,3,NaN,NaN,1,1
